<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Tuesday 7 August 2018</td>
<td style="text-align: right;"><a href="https://github.com/douglask3/savanna_fire_feedback_test">github.com/douglask3/savanna_fire_feedback_test</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity in climate potentials and mortality using Bayesian inference</h1>
<h2>Part 2: Bayesian inference</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>1 </sup>France Gerard, 
<sup>2 </sup>Rhys Whitley, 
<sup>3 </sup>Elmar Veenendaal
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<sup>3 </sup>Wageningen Univsersity, Wageningen UR
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to quantify the model parameters of a global tree cover model model. Tree Cover is calculated as a product  precipitation, tempurature  and radiation controls describing large scale climate potetials, and mortality from fire, seasonal drought, tempuature and people, and exlusion from land use modulate tree cover. This limitation model for tree cover (LimTREE) is optimized using a Bayesian Inference framework, which provides a probability distribution for the models paramters, and allows us to describe the magnitude and relative impact of controls and varaibles in terms of probabilities. Here, we cover model description and optimization.
</p>
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
</font>
</center>
<hr>
</div>

### Model description
Tree Cover ($T$) is calculated as a product five controls. Mean annual precipitation ($MAP$), mean annual tempurature ($MAT$) and incoming shortwave radiation ($SW$) represent three liberative controls on  $T$, i.e, increases in any of the variables lead to an increase in $T$. Mortality ($M$), which combines varaibles describing fire (represented by burnt area - $F$), drought (by number of dry days, $D$) and extreme seasonal tempuratures (by mean maximum temperuature of the hottest month, $MMxT$), and land use exclusion ($E$) from fractional urban ($U$), crop ($C$) and pasture ($P$) cover are suppresive controls. Each control is expressed as a linear combination of their respective variables and represented by a simple logistic curve:

\begin{equation}
    f(x) = \frac{1}{1 + e^{-k\cdot(x-x_0)}}
\end{equation}

\begin{eqnarray}
    T &=& F_{max} \cdot \prod f(x) \\[1em]
\end{eqnarray}

Where $f(x)$ is the limitation imposed by control $x = MAP, MAT, SW, M, E$ and $T_{max}$ is a maximum permitted monthly burnt area used to aid our model optimization. $x_0$ is the value of control $x$ when it imposes a limitation of 50\% on burnt area (i.e,$f(x) = 0.5$), and $k$ is the steepness of the logistic curve, equal to ¼ of the gradient at $x = x_0$. $k > 0$ for $MAP$, $MAT$ and $SW$, where $T$ increases with the control, whereas $k < 0$ for $M$ and $E$, for which $T$ decreases. Each suppresive control
is represented by a combination of variables ($x_i$) weighted by their respective influence ($v_i$). Where possible, units are consistent across variables within each control, and as such the combined variables are normalised to maintain these units:

\begin{eqnarray}
    x &=& \sum v_i \cdot x_i / \sum v_i  \\[1em]
\end{eqnarray}

\begin{eqnarray}
    v_i = 1
\end{eqnarray}



This leaves 14 free parameters that need to be optimised against observations of burnt area.

Now lets get this coded up in python

### Load libraries

In [1]:
import os
from   io     import StringIO
import numpy  as np
import pandas as pd
import csv

import pymc3  as pm3 
from   pymc3.backends import SQLite
from   scipy  import optimize
from   theano import tensor as tt

import matplotlib.pyplot as plt

# setup nice plotting
plt.style.use('ggplot')
%matplotlib inline

# paths and parameters
sample_pc     = 10
nIterations   = 20000 # 20000
nJobs         = 1    # 12
nChains       = 10   # 10
outPath       = "../data/vegFracControls.csv"
param_outpath = '../outputs/params-fullnewModel'

In [2]:
def npLog0 (x):
    return [np.log(i) if i > 0 else 0.0000001 for i in x]

## 2.1 Fire limitation model definition

Could possibly contain this in a class object, but I'm not sure theano can instantiate the object to be used by the GPU. If I've made absolutely no sense just then, then I would leave the following as is.

In [3]:
from pdb import set_trace as browser

def tt_log(x):
    x = tt.clip(x, 0.0000001, 1000000000)
    return tt.log(x)

def tt_sigmoid(x, k, x0):
    """
    Sigmoid function to describe limitation using tensor
    """
    return 1.0/(1.0 + tt.exp(-k*(x - x0)))


def MAP(map, drought, m_drought):
    """
    Definition to describe precip: while we are just returning input for now,
    we include climate potential functions for capability to be modified later.
    """
    map = map + (1.0-drought) * (1/m_drought) * (tt.exp(-m_drought * map) - 1)
    map = tt_log(map)
    return map


def SW(sw1, sw2, d):
    """
    Definition to describe short wave
    """
    return sw1 + d * sw2 /(1.0 + d)


def Temp(mat, min_mat, q10):
    """
    Definition to describe temp.
    """
    mat = q10**(mat - min_mat)
    
    #mat = mat - min_mat
    #mat = mat / (max_mat - min_mat)
    #mat = tt.clip(mat, 0.0000001, 100000)
    #mat = tt.log(mat)
    return mat

def convert_unity(x, k):
    return 1 - tt.exp(x * (-1.0/ k))

def mortality(fire, drought, maxTemp, minTemp, pop_density, 
              buffalo, cattle, goat, sheep, v_drought, v_maxTemp, v_minTemp,
              v_pop, v_buffalo, v_cattle, v_goat, v_sheep,
              p_fire, 
              k_pop, k_buffalo, k_cattle, k_goat, k_sheep,
              p_drought, min_maxTemp, max_minTemp, q10_maxTemp, q10_minTemp):
    """
    Definition of mortality
    """
    #maxTemp = maxTemp - min_maxTemp
    #maxTemp = maxTemp / (max_maxTemp - min_maxTemp)
    #maxTemp = tt.clip(maxTemp, 0.0, 1.0)
    #maxTemp = maxTemp **(p_maxTemp)
    
    maxTemp = Temp(maxTemp, min_maxTemp, q10_maxTemp)
    minTemp = Temp(-minTemp, -max_minTemp, q10_minTemp)
    
    drought = drought ** (p_drought)
    pop_density = convert_unity(pop_density, k_pop    )
    buffalo     = convert_unity(buffalo    , k_buffalo)
    cattle      = convert_unity(cattle     , k_cattle )
    goat        = convert_unity(goat       , k_cattle )
    sheep       = convert_unity(sheep      , k_sheep  )
    
    fire = fire**(p_fire)
    mort = fire  +  v_drought * drought  +  v_maxTemp * maxTemp  +  v_minTemp * minTemp + \
           v_pop * pop_density + v_buffalo * buffalo + v_cattle * cattle  + v_goat * goat + v_sheep * sheep

    return mort / (1.0 + v_drought + v_maxTemp + v_minTemp + v_pop + v_buffalo + v_cattle + v_goat + v_sheep)

def exclusion(urban_area, crop_area, v_crop):
    """
    Definition for the measure of fire supression
    """
    excl = urban_area  +  v_crop * crop_area
    
    return excl / (1.0 + v_crop)




## 2.2 Import data

Load data and do any necessary transformation needed for the Bayesian modelling framework. We  going to load 10% of data-points. When we evalutate, that gives us 90% spare.

In [4]:
def load_with_buffer(filename, line_select, **kwargs):
    s_buf = StringIO()
    line_select = np.sort(line_select)
    with open(filename) as file:
        count = -1
        lineN = -1
        for line in file:
            lineN += 1
            if lineN == 0 or lineN == line_select[count]:
                s_buf.write(line)
                count += 1
                if count == len(line_select): break
            
    s_buf.seek(0)
    df = pd.read_csv(s_buf,**kwargs)
    return df

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f): pass
    return i + 1

DATAPATH = os.path.expanduser(outPath)

nlines      = file_len(DATAPATH)
npoints     = round(sample_pc * nlines / 100)
line_select = np.random.choice(range(0, nlines), npoints, False)
fd          = load_with_buffer(DATAPATH, line_select)

Do a sanity check to make sure our data has imported correctly.

In [5]:
print(fd.info())
print(fd.head())
print(fd.min())
print(fd.max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 44 columns):
BurntArea                   2494 non-null float64
crop                        2494 non-null float64
MaxWind                     2494 non-null float64
buffalo                     2494 non-null float64
cattle                      2494 non-null float64
MADD_CMORPH                 2494 non-null float64
MADD_CRU                    2494 non-null float64
MADD_GPCC                   2494 non-null float64
MADD_MSWEP                  2494 non-null float64
MADM_CMORPH                 2494 non-null float64
MADM_CRU                    2494 non-null float64
MADM_GPCC                   2494 non-null float64
MADM_MSWEP                  2494 non-null float64
goat                        2494 non-null float64
MAP_CMORPH                  2494 non-null float64
MAP_CRU                     2494 non-null float64
MAP_GPCC                    2494 non-null float64
MAP_MSWEP                   2494 non-null f

## 2.3 Baysian framework

A simple explanation of Baye's law is:

\begin{equation}
    P(\beta|X) \propto P(\beta)\cdot P(X|\beta)
\end{equation}

where $X$ is our data (observations of some arbitrary system), and $\beta$ our set of unexplained parameters that describe the reponse of our _proposed understanding_ of this system as it varies with $X$.

### 2.3.1 Prior definitions
Because I have no idea what the uncertainty on the hyper parameters should look like (beyond $\beta> 0$), I've set them all as wide, familiar distrubtions, spread generously beyound what is realistic plausable. Our full prior looks like this:

\begin{eqnarray}
    P(x_0) &=& \mathcal{N}(0.5, 0.5) \\
    P(k_x) &=& \mathcal{exp}(0.1) \\
    P(T_{max}) = P(p_{popden}) = P(v_i) &=& \mathcal{exp}(1) \\
    P(\sigma) &=& \mathcal{N_{1/2}}(1) \\[1.5em]
\end{eqnarray}

So our full set of piors looks something like this:

\begin{eqnarray}
    P(\beta) &=& \prod_{x=1}^{4}\mathcal{S}(P(x_0),P(k_x), P(v_x)) \cdot P(D_{max}) \cdot P(\sigma) \\[1.5em]  
\end{eqnarray}

This is loosly describes a (albeit slightly skewed) Normal distribution, so we can approximate our error ($\sigma$) with a normal distibution. 

Back to the code.., `pymc3` is quite funky in that it allows me to create an empty `Model()` object and just add things to it as I need them using a `with` statement. I've called our Bayesian model `tree_error` as that is what we are trying to Quantify.



In [6]:
nChains = nChains * nJobs

In [7]:
def cutLastX(varname, mcmc_traces, ncut = 50):
    vals = mcmc_traces.get_values(varname)
    def subcut(vals, r, ncut = 50):
        cut_np = (r+1) * round(len(vals)/nChains)
        ncut = round(len(vals) * ncut / (nChains *100))
        return vals[(cut_np - ncut):cut_np]
    vals = [subcut(vals, r) for r in range(nChains)]
    return np.array(vals).flatten()

In [8]:
def run_pr_droughtVar(fire_dataset, pr_dataset, drough_var):
    with pm3.Model() as tree_error:

    # first for the sigmoids  
        MAP_x0         = pm3.Normal('MAP_x0'       ,  5.0, 1.0)
        MAP_k          = pm3.Exponential('MAP_k'        ,  1.0)

        MAT_x0         = pm3.Normal('MAT_x0'   ,  0.0, 1.0)
        MAT_k          = pm3.Exponential('MAT_k'    ,  1.0)

        SW_x0          = pm3.Normal('SW_x0'   ,  350, 350)
        SW_k           = pm3.Exponential('SW_k'    ,        1.5)

        mort_x0        = pm3.Normal('mort_x0', 0.5, 0.5)
        mort_k         = pm3.Exponential('mort_k' , 1.0)

        ex_x0          = pm3.Normal('ex_x0', 0.5, 0.5)
        ex_k           = pm3.Exponential('ex_k' , 1.0)

        max_T          = pm3.Uniform('max_T',     0.8)
        #pow_f          = pm3.Uniform('pow_f'         ,     0.0,    1.0)
    # now for the hyper-parameters that describe the independent fire condition covariates

        #cNpp = pm3.Uniform('cNPP', 0.0, 10)
        m_drought   = pm3.Exponential('m_drought', 1.0)
        v_drought   = pm3.Exponential('v_drought', 1.0)
        v_maxTemp   = pm3.Exponential('v_maxTemp', 1.0)
        v_minTemp   = pm3.Exponential('v_minTemp', 1.0)
        v_pop       = pm3.Exponential('v_pop'    , 1.0)
        v_buffalo   = pm3.Exponential('v_buffalo', 1.0)
        v_cattle    = pm3.Exponential('v_cattle' , 1.0)
        v_goat      = pm3.Exponential('v_goat'   , 1.0)
        v_sheep     = pm3.Exponential('v_sheep'  , 1.0)
        v_crop      = pm3.Exponential('v_crop'   , 1.0)

        trans_d     = pm3.Uniform('trans_d',  0.0, 1.0)
        p_fire      = pm3.Exponential('p_fire', 1.0)
        k_pop       = pm3.Exponential('k_pop' , 10.0)
        k_buffalo   = pm3.Exponential('k_buffalo', 1.0)
        k_cattle    = pm3.Exponential('k_cattle' , 1.0)
        k_goat      = pm3.Exponential('k_goat'   , 1.0)
        k_sheep     = pm3.Exponential('k_sheep'  , 1.0)

        p_drought   = pm3.Exponential('p_drought', 1.0)
        min_maxTemp = pm3.Normal     ('min_maxTemp', 20.0, 31.34)
        max_minTemp = pm3.Normal     ('max_minTemp', 30.0, 31.34)
        min_mat     = pm3.Normal     ('min_mat', 0.0, 3.0)
        max_mat     = pm3.Normal     ('max_mat', 15.0, 5.0)
        q10_maxTemp = pm3.Exponential('q10_maxTemp', 1.0)
        q10_minTemp = pm3.Exponential('q10_minTemp', 1.0)
        
        # describe the standard deviation in the error term
        sigma       = pm3.HalfNormal('sigma', sd=1)


        # transform hyper-covariates 
        MAP_var = 'MAP_' + pr_dataset
        drght_var = drough_var + '_' + pr_dataset
        fire_var = "BurntArea_" + fire_dataset
        
        
        f_precip = MAP(fd[MAP_var].values, fd[drght_var].values, m_drought)
        #f_temp   = MAT(fd["MAT"].values, min_mat, max_mat)
        f_sw     = SW (fd["SW"].values, fd["SW.1"].values, trans_d)
        
        f_mort   = mortality(fd[fire_var].values, fd[drght_var].values, fd["MTWM"].values, fd["MAT"].values, 
                             fd["PopDen"].values, fd["buffalo"].values, fd["cattle"].values, fd["goat"].values,
                             fd["sheep"].values,
                             v_drought, v_maxTemp, v_minTemp,
                             v_pop, v_buffalo, v_cattle, v_goat, v_sheep,
                             p_fire,
                             k_pop, k_buffalo, k_cattle, k_goat, k_sheep,
                             p_drought,
                             min_maxTemp, max_minTemp, q10_maxTemp, q10_minTemp)
                                
        f_exc    = exclusion(fd["urban"].values, fd["crop"].values, v_crop)


        # Tree cover is assumed to be the product of the 4 sigmoid

        prediction = max_T * np.product([tt_sigmoid(f_precip, MAP_k, MAP_x0),
                                 #tt_sigmoid(f_temp, MAT_k, MAT_x0),
                                 tt_sigmoid(f_sw, SW_k, SW_x0),
                                 tt_sigmoid(f_mort, - mort_k, mort_x0),
                                 tt_sigmoid(f_exc, - ex_k, ex_x0)])

        #prediction = tt.clip(prediction, -100, 0.8)              
        # calculate the error between observed and predicted burnt area
        error = pm3.Normal('error', mu = prediction, sd = sigma, observed=fd['TreeCover'].values)

         # set the step-method (criteria algorithm for moving around information space)
        step = pm3.Metropolis()

        # do the sampling
        mcmc_traces = pm3.sample(nIterations, step=step, chains = nChains) 
    
    #pm3.traceplot(mcmc_traces);
    varnames = mcmc_traces.varnames
    
    vals = [cutLastX(i, mcmc_traces) for i in varnames]

    vals = pd.DataFrame(np.array(vals).T, columns=varnames)
    out_fname = param_outpath + '_' + fire_dataset + '_' + pr_dataset + '_' + drough_var + '.csv'
    vals.to_csv(out_fname, index=False)
    #plt.plot(vals["MAP_x0"])
    return mcmc_traces
    

In [ ]:
for fire_d in ["GFED_four_s", "GFED_four", "MCD_forty_five", "meris", "MODIS"]:
    for pr_d in ['MSWEP', 'GPCC', 'CRU', 'CMORPH']: 
        for dr_v in ['MADD', 'MADM', 'MConc', 'MDDM']:
            mcmc_traces = run_pr_droughtVar(fire_d, pr_d, dr_v)

Multiprocess sampling (10 chains in 4 jobs)
CompoundStep
>Metropolis: [sigma]
>Metropolis: [q10_minTemp]
>Metropolis: [q10_maxTemp]
>Metropolis: [max_mat]
>Metropolis: [min_mat]
>Metropolis: [max_minTemp]
>Metropolis: [min_maxTemp]
>Metropolis: [p_drought]
>Metropolis: [k_sheep]
>Metropolis: [k_goat]
>Metropolis: [k_cattle]
>Metropolis: [k_buffalo]
>Metropolis: [k_pop]
>Metropolis: [p_fire]
>Metropolis: [trans_d]
>Metropolis: [v_crop]
>Metropolis: [v_sheep]
>Metropolis: [v_goat]
>Metropolis: [v_cattle]
>Metropolis: [v_buffalo]
>Metropolis: [v_pop]
>Metropolis: [v_minTemp]
>Metropolis: [v_maxTemp]
>Metropolis: [v_drought]
>Metropolis: [m_drought]
>Metropolis: [max_T]
>Metropolis: [ex_k]
>Metropolis: [ex_x0]
>Metropolis: [mort_k]
>Metropolis: [mort_x0]
>Metropolis: [SW_k]
>Metropolis: [SW_x0]
>Metropolis: [MAT_k]
>Metropolis: [MAT_x0]
>Metropolis: [MAP_k]
>Metropolis: [MAP_x0]
Sampling 10 chains:  41%|████      | 83505/205000 [41:49<1:00:24, 33.52draws/s]

### 2.3.2 Likelihood definition

For the sake of simplicity (and because I don't really know any better), we define the model error as normally distributed (i.i.d.) although it most likely isn't. We could make this more complicated later by defining the error as heteroscedastic, but I wouldn't bother with that until we have some idea of the convergence. We're describing the error (observations minus model predictions) as follows:

\begin{eqnarray}
    P(X|\beta) &=& \mathcal{N}(F_{burn}, \sigma) \\[1em]
    \mathcal{N}(F_{burn}, \sigma) &=& \frac{N}{\sigma\sqrt{2\pi}}\exp\left\{\sum_{i=1}^{N}\left(\frac{y_i - F_{burn, i}}{\sigma_i}\right)^2\right\}
\end{eqnarray}

where $y_i$ is a set of observations we're attempting to optimise on. Below is the code that describes the above:

### 2.3.3 Posterior sampling

Because it is nigh impossible to determine the posterior solution analytically we will instead sample the information space to **infer** the posterior solutions for each of the model parameters. In this case we are using a Metropolis-Hasting step MCMC.

I've tried using No-U-Turn (NUTS) sampling (which is the new kid on the block), but there are issues with it's current implementation in pymc3 (see github repo issues). Can use it once problems are ironed out - but TBH it doesn't matter if we're getting a reasonable convergence.

In [ ]:
#pm3.traceplot(mcmc_traces);

The iterations at the start are just letting the optimization settle. So we will only sample to last 5% of iterations for futher analysis. Here, exporting to netcdf for others to do their own analysis

In [ ]:
print(param_outpath + '.csv')

In [ ]:
def cutLastX(varname, ncut = 50):
    vals = mcmc_traces.get_values(varname)
    def subcut(vals, r, ncut = 50):
        cut_np = (r+1) * round(len(vals)/nChains)
        ncut = round(len(vals) * ncut / (nChains *100))
        return vals[(cut_np - ncut):cut_np]
    vals = [subcut(vals, r) for r in range(nChains)]
    return np.array(vals).flatten()

varnames = mcmc_traces.varnames

vals = [cutLastX(i) for i in varnames]

vals = pd.DataFrame(np.array(vals).T, columns=varnames)
vals.to_csv(param_outpath + '.csv', index=False)

vals.head()
plt.plot(vals["mort_x0"])

## Plotting

Let look at the pdf of the last 5% of iterations for each parameter

In [ ]:
variables2Plot = varnames

nvar = len(variables2Plot)
nprow = 4
npcol = np.ceil(nvar / nprow)

plt.figure(figsize=(5*nprow,5 * npcol))
def plotVar(var1, pn):
    plt.subplot(npcol, nprow, pn)
    param = vals[var1]
    
    hist, bins = np.histogram(param, bins=50)
    hist = 100.0 * hist / np.sum(hist)
    bins = bins[1:] - np.diff(bins)/2
    plt.plot(bins, hist)
    plt.xlabel(var1)
    
pn = 0
for i in variables2Plot:
    pn = pn + 1
    plotVar(i, pn)

And what to the sigmoids look like?

In [ ]:
from pdb import set_trace as browser

def pltVsFire(x, xlab, pnt = 'o', *args, **kw):
    plt.plot(x, fd.fire, pnt, alpha = 0.03, *args, **kw)
    plt.xlabel(xlab)
    
def np_sigmoid(x, k, x0):
    """
    Sigmoid function to describe limitation using tensor
    """
    return 1.0/(1.0 + np.exp(-k*(x - x0)))

def returnSigmoid(x, k, x0):
    return np_sigmoid(x, k, x0)
    
def meanParam(x, x0, k, kmult = 1.0):
    x0 = np.mean(vals[x0])
    k  = np.mean(vals[k]) * kmult

    return returnSigmoid(x, k, x0)

def randomParam(x, x0, k, kmult = 1.0, size = 100):
    ps = np.random.choice(vals.shape[0], size = size, replace = False)
    return [returnSigmoid(x, vals[k][i] * kmult, vals[x0][i]) for i in ps]

In [ ]:
plt.figure(figsize=(10, 10))

####################
## Fuel           ##
####################
plt.subplot(2, 2, 1)
## scatter plot
fp = np.mean(vals['fuel_pw'])
fg = np.mean(vals['fuel_pg'])
cNPP = 1.0#np.mean(vals['cNPP'])

f_fuel = fuel_load(fd["vegCover"].values, fd["alphaMax"].values, fp, fg)
print(f_fuel.max())
pltVsFire(f_fuel, "NPP (g/m$^2$)", 'go')

## Line of best fit
#Fuel = np.arange(-6, fd.fuel.max(), 0.01)
Fuel = np.arange(0, f_fuel.max(), 0.01)
r_fuel = randomParam(Fuel, 'fuel_x0', 'fuel_k')
#NPP = np.exp(Fuel)
for r in r_fuel: plt.plot(Fuel, r, 'k', alpha=.01)

## cfg plot
#plt.xscale('log')
#plt.xlim([0.5, fd.NPP.max()])

####################
## Moisture       ##
####################
plt.subplot(2, 2, 2)
## scatter plot
Fmax = np.mean(vals['max_f'])
cM = np.mean(vals['cM'])
cMT = np.mean(vals['cMT'])
f_moisture = moisture(fd["alpha"].values, fd["emc"].values, fd["treecover"].values, cM, cMT)
pltVsFire(f_moisture , "Moisture = $\\alpha$ + M $\cdot$ EMC",'bo')

## Line of best fit
mst = np.arange(0.0, f_moisture.max(), 0.05)
r_moisture = randomParam(mst, 'moisture_x0', 'moisture_k', -1.0)
for r in r_moisture: plt.plot(mst, r, 'k', alpha=.01)

    
plt.ylim(0, 1)

####################
## Igntions       ##
####################
plt.subplot(2, 2, 3)
## scatter plot 
cP  = np.mean(vals['cP1'])
cC  = np.mean(vals['cC' ])
cD1 = np.mean(vals['cD1'])
igniteMax = 10

f_ignite = ignition(fd["lightning_ignitions"].values, \
                    fd["cropland"].values, \
                    fd["pasture"].values, \
                    fd["population_density"].values, \
                    cC, cP, cD1)
#f_ignite = np.exp(f_ignite)
pltVsFire(f_ignite, "Ignitions events = Lightn + P $\cdot$ Pop Dens + D1 $\cdot$ Pasture")

## Line of best fit
Ignite = np.arange(0.0, igniteMax, 0.1)
#Ignite = np.exp(Ignite)
#=Ignite = np.arange(0.0, 10000, 1)
r_Ignite = randomParam(Ignite, 'igntions_x0', 'igntions_k')
#yay = returnSigmoid(Ignite, 10, 1.0)
#Ignite = np.exp(Ignite)
for r in r_Ignite: plt.plot(Ignite, r, 'k', alpha=.01)

#plt.plot(Ignite, yay, 'red')

    
## cfg plot
#plt.xscale('log')
plt.xlim(0, igniteMax)

####################
## Suppression    ##
####################
plt.subplot(2, 2, 4)
#scatter plot
cP2 = np.mean(vals['cP2'])
cD2 = np.mean(vals['cD2'])
maxF = np.mean(vals['max_f'])
f_suppression = maxF * supression(fd["cropland"].values, \
                           fd["cropland"].values, \
                           fd["population_density"].values, \
                           cP2,
                           cD2)

pltVsFire(f_suppression, "Suppression = Cropland + D2 $\cdot$ Pop den")

# Line of best fit
Suppress = np.arange(0, 100, 0.01)
r_suppression = randomParam(Suppress, 'suppression_x0', 'suppression_k', -1.0)
for r in r_suppression: plt.plot(Suppress, r * Fmax, 'k', alpha=.01)

plt.xlim(0, 100)
plt.ylim(0, 1)